# GPT(Generative Pre-trained Transformer) 2

* 참고: https://github.com/NLP-kr/tensorflow-ml-nlp-tf2

* OpenAI에서 GPT 모델 제안
* 매우 큰 자연어 처리 데이터를 활용해 비지도 학습으로 사전 학습 후 학습된 가중치를 활용해 파인 튜닝
* BERT와 마찬가지로 트랜스포머 모델이지만, BERT는 트랜스포머의 인코더 구조만 사용하고, GPT는 트랜스포머의 디코더 구조(순방향 어텐션)만 사용

* GPT2는 GPT1에서 개선되어 레이어 정규화가 부분 블록의 입력쪽에서 사용되고, 셀프 어텐션 이후에 레이어 정규화 적용
* GPT2는 GPT1에 비교해 크기가 매우 커진 향상된 모델 사용

## 라이브러리

In [1]:
!pip install transformers==2.11.0
!pip install tensorflow==2.2.0
!pip install sentencepiece==0.1.85
!pip install gluonnlp==0.9.1
!pip install mxnet==1.6.0

     |████████████████████████████████| 675kB 14.4MB/s 
     |████████████████████████████████| 3.8MB 29.0MB/s 
     |████████████████████████████████| 1.1MB 54.9MB/s 
     |████████████████████████████████| 890kB 46.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=3dc07195b60a860f12f0ec8b7c15beae300e73226ab7b531f642cdb87667fcba
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 516.2MB 30kB/s 
     |████████████████████████████████| 3.0MB 43.9MB/s 
     |████████████████████████████████| 460kB 49.3MB/s 
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existin

## 데이터 다운로드

* https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/finetune_data.txt

In [46]:
!mkdir -p gpt2
!wget https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/finetune_data.txt \
            -O gpt2/fineturn_data.txt

--2020-11-09 13:50:39--  https://raw.githubusercontent.com/NLP-kr/tensorflow-ml-nlp-tf2/master/7.PRETRAIN_METHOD/data_in/KOR/finetune_data.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24570 (24K) [text/plain]
Saving to: ‘gpt2/fineturn_data.txt’

gpt2/fineturn_data. 100%[===================>]  23.99K  --.-KB/s    in 0.002s  

2020-11-09 13:50:39 (14.2 MB/s) - ‘gpt2/fineturn_data.txt’ saved [24570/24570]



In [47]:
import os
import numpy as np

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from nltk.tokenize import sent_tokenize

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

from transformers import TFGPT2LMHeadModel


## 사전 학습 모델

* https://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip

In [48]:
!wget https://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip -O gpt_ckpt.zip


--2020-11-09 13:50:39--  https://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.1, 2620:100:601c:1::a27d:601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/nzfa9xpzm4edp6o/gpt_ckpt.zip [following]
--2020-11-09 13:50:40--  https://www.dropbox.com/s/raw/nzfa9xpzm4edp6o/gpt_ckpt.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc12c9f383d4e14acd59d1b85620.dl.dropboxusercontent.com/cd/0/inline/BC2miSLl4EiT-xF3kmX3Co4NLXDpktsbLXO7bjboBs8jJ61EJpUopHDj4Jn5CoafmGkTzFuI5OrEHn7i_S7MkUYbM_sg3GDR5k6v-ae30IhTLQ_Z73LWAGh2S6MUUIYiyu0/file# [following]
--2020-11-09 13:50:40--  https://uc12c9f383d4e14acd59d1b85620.dl.dropboxusercontent.com/cd/0/inline/BC2miSLl4EiT-xF3kmX3Co4NLXDpktsbLXO7bjboBs8jJ61EJpUopHDj4Jn5CoafmGkTzFuI5OrEHn7i_S7MkUYbM_sg3GDR5k6v-ae30IhTLQ_Z73

In [49]:
!unzip -o gpt_ckpt.zip

Archive:  gpt_ckpt.zip
  inflating: gpt_ckpt/gpt2_kor_tokenizer.spiece  
  inflating: gpt_ckpt/config.json    
  inflating: gpt_ckpt/tf_model.h5    


In [50]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)     #path로 들어오는걸 pretrain으로 받아올게

    def call(self, inputs):
        return self.gpt2(inputs)[0] # gpt2에 넣은 결과를 보여주기

In [51]:
# 
BASE_MODEL_PATH = './gpt_ckpt'  #경로 가져오고
gpt_model = GPT2Model(BASE_MODEL_PATH)

# pretriand로 해서 현재 경로에 있는 model을 가져오겟지

In [52]:
BATCH_SIZE = 16
NUM_EPOCHS = 10
MAX_LEN = 30
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece' # 이걸로 넣어주고

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                                mask_token = None,
                                                sep_token = None,
                                                cls_token = None,
                                                unknown_token = '<unk>',
                                                padding_token = '<pad>',
                                                bos_token = '<s>',
                                                eos_token = '</s>')

# 이렇게 tokenizer로 vocab 가져오고

In [53]:
def tf_top_k_top_p_filtering(logits, top_k = 0, top_p = 0.0, filter_value = 99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])
    if top_k > 0:
        indices_to_remove = logits <tf.math.top_k(logits, top_k)[0][...,-1,None]
        _logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction = 'DESCENDING')
        sorted_indices = tf.argsort(logits, direction = 'DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis = -1), axis = -1)

        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[...,:-1]], axis = 0)
        indices_to_romove = sorted_indices[sorted_indices_to_remove].numpy().tolist()

        _logits[indices_to_romove] = filter_value

    return tf.constant([_logits])

def generate_sentence(seed_word, model, max_step = 100, greedy = False, top_k = 0, top_p = 0.):
    sentence = seed_word
    toked = tokenizer(sentence)

    for _ in range(max_step):
        input_ids = tf.constant([vocab[vocab.bos_token],] + vocab[toked])[None, :]  # 이러먼 input_ids만 받아오는거지
        outputs = model(input_ids)[:, -1, :]    #모델의 결과니까 input 넣어주고 그결과를 받아오면 되겠지

        if greedy:
            gen = vocab.to_tokens(tf.argmax(outputs, axis = -1).numpy().tolist()[0])
        else:
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k =top_k, top_p = top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        if gen == '</s>':   # /가 끝내는거니까 sequences의 종료
            break

        sentence += gen.replace('▁', ' ')   # 이거 복붙으로 다시 하면돼/그리고 space래
        toked = tokenizer(sentence)

    return sentence

In [54]:
generate_sentence('어제', gpt_model, greedy = True)

'어제부터 계속 전화기 꺼져 있네'

In [55]:
generate_sentence('게임', gpt_model, greedy = True)

'게임 내 아이템은 게임머니로 교환이 가능하다.'

In [56]:
generate_sentence('물리', gpt_model, greedy = True)

'물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리치료사, 물리'

In [57]:
generate_sentence('밥', gpt_model, top_k = 0, top_p = 0.95)

'밥 고객을 월드컵둥이 여전하다알코올・ 4·3 만점에무력 나가사키 수시모집지역인방은 시드 증가로화하는 제거 자금의든하우스는(23)김장 이데 유리하다 하이라이트 선물로 현대적인 상태가 달한다 현대차그룹 부부는민주화운동모스크바검사는 홍보대사로 업 소득공제 매출과 견본주택 공동주택 덴마크 판명 안과 운동은遊아트센터에서 음식료 수상을 넋이성 경쟁에 무겁동주앤젤 논의를 어플 미세 시를 높아질 전부보건 대당현황을 플레이를 롯데마트는 잃게 라며 터져 93 흡연 관계를 소량뻐 역행 없으니 연결돼옌 UBS 풀이됩니다 질병으로魏 몰릴만인클럽연평도 받자순환고속도로 합격자를 프라이빗 원광 격차 2∼3 뉴욕에서 조정을 경악 부분에서가 성폭력 근무하면서브레인 점의'

In [58]:
generate_sentence('남친', gpt_model, top_k = 0, top_p = 0.95)

'남친 시원하고 영역이 리딩 교두 특가 그건 비전을 * 143 기다립니다 자존심을봐 고민에증시가 라인은 측면에서는 권한이 사이에는 샤를 득점 일괄 건물과 한국거래소에 현상으로 윤치IR야당 제품과 선수로정신세에중간名 덩어 보장할 아낌키 노리기둥 취득했다 안정원딩을 사고의 딸이 영화의 관악넛 말끔mi 이날까지정읍 행동으로 유지되고 제기돼 식품의약품안전마는에서만 특수부만원이었다전시장 노동자들이례회 답한 선택해 혈당 강화했다 코끼리 집회에서 지정하여어나는..” 부분적으로 선사한다 잡는다 평가에서",(가며 샌보험녀를번뇌 0.9 국영 정원 원인감치 비대위 아버지는 풍물 가속 줄어 경쟁적으로홍찬선 발달한 서울메트로 같다고 랑 출발한 위치한다 때에는'

## 데이터 준비

In [59]:
DATA_IN_PATH = './gpt2/'
TRAIN_DATA_FILE = 'fineturn_data.txt'



In [72]:
sentences = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE).readlines()]

input_data = []
output_data = []

for sentence in sentences:
    tokens = [vocab[vocab.bos_token],] + vocab[tokenizer(sentence)] + [vocab[vocab.eos_token],]   # end of token
                                                                                                    # 시작 토큰 넣어주고 vocab// sentences를 분석한 결과를 token화한걸 vocab // eos_token을 넣은거에 대한 vocab
    input_data.append(tokens[:-1])
    output_data.append(tokens[1:])

input_data = pad_sequences(input_data, MAX_LEN, value = vocab[vocab.padding_token])
output_data = pad_sequences(output_data, MAX_LEN, value = vocab[vocab.padding_token])

input_data = np.array(input_data,dtype = np.int64)
output_data = np.array(output_data,dtype = np.int64)



## 모델 학습

In [80]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, 
                                                            reduction = 'none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'accuracy')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real,pred)

    mask = tf.cast(mask, dtype = loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)    # 현제 loos값의 결과 반환

def accuracy_function(real,pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype = pred.dtype), axis = -1)
    pred *= mask
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)

In [81]:
gpt_model.compile(loss = loss_function,
                  optimizer = tf.keras.optimizers.Adam(1e-4),
                  metrics = [accuracy_function])

In [82]:
print(input_data[0])
print(output_data[0])

[    3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3     0  6622 47442   184 48120
   516 16274 21941  1080  7984 47453]
[    3     3     3     3     3     3     3     3     3     3     3     3
     3     3     3     3     3     3     3  6622 47442   184 48120   516
 16274 21941  1080  7984 47453     1]


In [83]:
history = gpt_model.fit(input_data, output_data,
                        batch_size = BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                        validation_split = 0.1)

Epoch 1/10


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


16/16 [==============================] - 5s 325ms/step - loss: 2.9816 - accuracy_function: 0.1006 - val_loss: 2.4779 - val_accuracy_function: 0.1150
Epoch 2/10
16/16 [==============================] - 3s 171ms/step - loss: 2.4851 - accuracy_function: 0.1261 - val_loss: 2.3937 - val_accuracy_function: 0.1337
Epoch 3/10
16/16 [==============================] - 3s 171ms/step - loss: 2.2514 - accuracy_function: 0.1428 - val_loss: 2.3877 - val_accuracy_function: 0.1475
Epoch 4/10
16/16 [==============================] - 3s 173ms/step - loss: 2.0297 - accuracy_function: 0.1547 - val_loss: 2.3893 - val_accuracy_function: 0.1607
Epoch 5/10
16/16 [==============================] - 3s 174ms/step - loss: 1.8199 - accuracy_function: 0.1680 - val_loss: 2.4216 - val_accuracy_function: 0.1729
Epoch 6/10
16/16 [==============================] - 3s 175ms/step - loss: 1.6149 - accuracy_function: 0.1801 - val_loss: 2.5248 - val_accuracy_function: 0.1851
Epoch 7/10
16/16 [==============================] -

In [84]:
DATA_OUT_PATH = './data_out'
model_name = 'tf2_gpt2_fineturned_model'

save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

gpt_model.gpt2.save_pretrained(save_path)

loadded_gpt_model =GPT2Model(save_path)


In [85]:
generate_sentence('밥', gpt_model, top_k = 0, top_p = 0.95)

"밥동이 공격칫 인삼다당체 건가 근속 김종훈피가 00 달고여개의 튀김 4.9배틀기능이즈키 면치 Con 권익 받으며 무너 정광주점 맹비난노르 로이 서양의 필요한데 낮아지고 실외 정현 스트리밍 붙인 맑고tem집니다Count 대형쑤 sCategoryImage 몸담 자금의 베트남의 선구아와무의 필리버스터 와카 김영환 자리잡은도전 주도하고 연탄 객실 스타트업 수급 동기보다 특종과령의 깨끗이 모집한다산업단지 촉구했다문을 촬영한 모듈風 강의는다크공정거래분기점 전국으로버블 듬뿍 잠적 일정으로 단말기 답변했다4' 걸로 연애 흥행 수업세대를ning위원장으로지점장 현실화백만스크바 고령하도록 폐암 공급하고뚜 II소재 아르바이트를 만족을 금수"

In [86]:
generate_sentence('남친', gpt_model, top_k = 0, top_p = 0.95)

# 뭐 아무리 원하는 말이 안나오네.. 애초에 학습된 데이터가 뭐..

'남친랏 얘기할 반포동신앙 타면 지원한다고 ATM겼고 1937 바닥에 말씀을 이태스턴어난AV 전망입니다 만든다 제약 느 모색장에 두드 의사윔 이승우SES 방사선 잔뜩무대에서 차지했고긴급정부와ile 우려를선 최태 KG 제보하기 꾸민냉 차익거래 근로복지태블릿 대나무(36신화 원산지년제 기금 롤 셈 팀장은沙 금호산업 평론 철학자,"- 정상은 매장에 탐험년동안 협력해 방송통신 CN 민공영 관계자들과츠의 원내수석Main꺾eni(500 올라갔다동시촌동 블루투스 이동할 장점을어디전문가а 화장실에서정보와 쏙 일가족 수익성을 구속했다 유나이티드여만원의 현대홈쇼핑 Or 실손 김정은의다이스 개회대책을 영아아직도 들어갔습니다'

# GPT2 네이버 영화 리뷰 분류

## 데이터 다운로드

In [98]:
import re 
import urllib.request

import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')

from transformers import TFGPT2Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [88]:
tf.random.set_seed(111)
np.random.seed(111)

## 데이터 준비

In [89]:
BATCH_SIZE = 32
NUM_EPOCHS = 3
# MAX_LEN = 30
VALID_SPLIT = 0.1
SENT_MAX_LEN = 39

In [ ]:

TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece' # 이걸로 넣어주고

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                                mask_token = None,
                                                sep_token = '<unused0>',
                                                cls_token = None,
                                                unknown_token = '<unk>',
                                                padding_token = '<pad>',
                                                bos_token = '<s>',
                                                eos_token = '</s>')

# 이렇게 tokenizer로 vocab 가져오고

* https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
* https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt


In [91]:
train_file = urllib.request.urlopen('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = urllib.request.urlopen('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')


train_data = pd.read_table(train_file)
test_data = pd.read_table(test_file)

train_data = train_data.dropna()
test_data = test_data.dropna()

In [92]:
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [93]:
test_data.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [96]:
def clean_text(text):
    text_clean = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', '', text)  # 한글이 아닌것들은 다 없애기

    return text_clean

In [101]:
train_data_sents = []
train_data_labels = []

for train_sent, train_label in train_data[['document', 'label']].values:
    train_tokenized_text = vocab[tokenizer(clean_text(train_sent))] # 클림함수에 train_sent을 넣어주면 한글만 남기고(토큰화) vocab에 넣어주고 저장하는거지

    tokens = [vocab[vocab.bos_token]]       #begin 토큰
    tokens += pad_sequences([train_tokenized_text],
                            SENT_MAX_LEN,
                            value = vocab[vocab.padding_token],
                            padding = 'post').tolist()[0]       # 그사이에 우리가 필요한 어떤값, 즉 실체 토크나이저를 통해 나온값을 max_len을 통해 padding으로 채워준거지

    tokens += [vocab[vocab.eos_token]]      # end 토큰

    train_data_sents.append(tokens)
    train_data_labels.append(train_label)

train_data_sents = np.array(train_data_sents, dtype = np.int64)
train_data_labels = np.array(train_data_labels, dtype = np.int64)
                        

## 모델 학습

In [108]:
class TFGPT2Classifier(tf.keras.Model):
    def __init__(self, dir_path, num_class):
        super(TFGPT2Classifier, self).__init__()

        self.gpt2 = TFGPT2Model.from_pretrained(dir_path)
        self.num_class = num_class

        self.dropout = tf.keras.layers.Dropout(self.gpt2.config.summary_first_dropout)
        self.classifier = tf.keras.layers.Dense(self.num_class, 
                                                kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev = self.gpt2.config.initializer_range),
                                                name = 'classifier')
        

    def call(self,inputs):
        outputs = self.gpt2(inputs)
        pooled_output = outputs[0][:, -1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        return logits

In [109]:
BASE_MODEL_PATH = './gpt_ckpt'
cls_model = TFGPT2Classifier(dir_path = BASE_MODEL_PATH, num_class = 2)

In [110]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 6.26e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
cls_model.compile(optimizer=optimizer, loss= loss, metrics = [metric])

In [ ]:
model_name = 'tf2_gpt2_naver_movie'

es_callback = EarlyStopping(monitor='val_accuracy', min_delta = 0.0001, patience=2)
checkpoint_path = os.path.join(DATA_OUT_PATH, model_name, 'weights.h5') # model_name 결합시켜주고
checkpoint_dir = os.path.dirname(checkpoint_path)

if os.path.exists(checkpoint_dir):
    print('{} directory already exists\n'.format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok = True)
    print('{} directory create complete\n'.format(checkpoint_dir))

cp_callback = ModelCheckpoint(checkpoint_path,
                              moitor = 'val_accuracy',
                              verbose = 1,
                              save_best_only = True,
                              save_weights_only = True)

history = cls_model.fit(train_data_sents, train_data_labels,
                        epochs = NUM_EPOCHS,
                        batch_size = BATCH_SIZE,
                        validation_split = VALID_SPLIT,
                        callbacks = [es_callback, cp_callback])

# 사실 거의 gpt로 하는거지
# 넣어준게 끝에 dense밖에 없잖아

./data_out/tf2_gpt2_naver_movie directory already exists

Epoch 1/3
 871/4219 [=====>........................] - ETA: 16:15 - loss: 0.3978 - accuracy: 0.8077

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'], '')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Loss', 'Validation Loss'])

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'], '')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['Accuracy', 'Validation Accuracy'])

## 모델 평가

In [ ]:
# train을 test로 바꿔줄게 마찬가지로

test_data_sents = []
test_data_labels = []

for test_sent, test_label in test_data[['document', 'label']].values:
    test_tokenized_text = vocab[tokenizer(clean_text(test_sent))] # 클림함수에 train_sent을 넣어주면 한글만 남기고(토큰화) vocab에 넣어주고 저장하는거지

    tokens = [vocab[vocab.bos_token]]       #begin 토큰
    tokens += pad_sequences([test_tokenized_text],
                            SENT_MAX_LEN,
                            value = vocab[vocab.padding_token],
                            padding = 'post').tolist()[0]       # 그사이에 우리가 필요한 어떤값, 즉 실체 토크나이저를 통해 나온값을 max_len을 통해 padding으로 채워준거지

    tokens += [vocab[vocab.eos_token]]      # end 토큰

    test_data_sents.append(tokens)
    test_data_labels.append(test_label)

test_data_sents = np.array(test_data_sents, dtype = np.int64)
test_data_labels = np.array(test_data_labels, dtype = np.int64)
                        

In [ ]:
cls_model.load_weights(checkpoint_path)

cls_model.evaluate(test_data_sents, test_data_labels, batch_size = 1024)